In [ ]:
import random
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm
from datetime import datetime

In [ ]:
def load_annotations(annotation_file):
    with open(annotation_file, 'r') as f:
        return json.load(f)

In [ ]:
def get_image_annotations(annotations, image_id):
    return [ann for ann in annotations if ann['image_id'] == image_id]

In [ ]:
def draw_bbox(ax, bbox, color='r'):
    x, y, w, h = bbox
    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

In [ ]:
def draw_bbox(ax, bbox, color='r'):
    x, y, w, h = bbox
    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

In [ ]:
def visualize_bbox(original_folder, sr_folder, annotation_file, num_samples=4):
    annotations = load_annotations(annotation_file)
    
    print(f"Total images in annotations: {len(annotations['images'])}")
    
    # 처음 4개의 이미지 선택
    selected_images = annotations['images'][:num_samples]
    print(f"Selected {len(selected_images)} images for visualization")
    
    fig, axs = plt.subplots(len(selected_images), 2, figsize=(20, 5*len(selected_images)))
    
    for idx, img_info in enumerate(selected_images):
        original_filename = img_info['file_name']
        image_id = img_info['id']
        
        # 원본 이미지 경로
        original_path = os.path.join(original_folder, original_filename)
        
        # 확대된 이미지 경로 (파일명 변경)
        sr_filename = original_filename.replace('.jpg', '_x2_SR.png')
        sr_path = os.path.join(sr_folder, sr_filename)
        sr_path = os.path.splitext(sr_path)[0] + '.png'
        
        # 이미지 로드
        original_img = Image.open(original_path)
        sr_img = Image.open(sr_path)
        
        # 어노테이션 가져오기
        image_anns = get_image_annotations(annotations['annotations'], image_id)
        
        # 원본 이미지에 bbox 그리기
        axs[idx, 0].imshow(original_img)
        axs[idx, 0].set_title(f'Original Image: {original_filename}')
        for ann in image_anns:
            draw_bbox(axs[idx, 0], ann['bbox'])
        axs[idx, 0].axis('off')
        
        # 확대된 이미지에 bbox 그리기
        axs[idx, 1].imshow(sr_img)
        axs[idx, 1].set_title(f'2x Scaled Image: {sr_filename}')
        for ann in image_anns:
            bbox = ann['bbox']
            # bbox 좌표를 2배로 조정
            scaled_bbox = [coord * 2 for coord in bbox]
            draw_bbox(axs[idx, 1], scaled_bbox, color='r')
        axs[idx, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

### SR을 이용한 해상도 2배 이미지 bbox 체크하기

In [ ]:
# 함수 실행
original_folder = '/data/ephemeral/home/dataset/'  # 원본 이미지 폴더
sr_folder = '/data/ephemeral/home/EDSR-PyTorch/experiment/test/results-Demo/'  # 확대된 이미지 폴더
annotation_file = '/data/ephemeral/home/dataset/train.json'

visualize_bbox(original_folder, sr_folder, annotation_file)

### 2배 해상도 이미지에서 4분할 하기

In [ ]:
x2_annotation_path = '/data/ephemeral/home/dataset/train_x2.json'

# Read json
with open(annotation_file, 'r') as file:
    data = json.load(file)

# Modify bbox values
for ann in data['annotations']:
    ann['bbox'][0] *= 2
    ann['bbox'][1] *= 2
    ann['bbox'][2] *= 2
    ann['bbox'][3] *= 2

# Store new json file
with open(x2_annotation_path, 'w') as file:
    json.dump(data, file, indent=2)

In [ ]:
# Dataset path
dataDir = '/data/ephemeral/home/EDSR-PyTorch/experiment/test/results-Demo'
annotation_path = '/data/ephemeral/home/dataset/train_x2.json'
original_anno_path = '/data/ephemeral/home/dataset/train.json'

# saved path
subimgs_path = '/data/ephemeral/home/dataset/scale_x2'
updated_annotation_path = '/data/ephemeral/home/dataset/train_x2.json'

In [ ]:
def update_annotations_for_subimage(annotations, subimg_info, img_id):
    updated_annotations = []
    x_offset, y_offset, subimg_width, subimg_height = subimg_info

    for ann in annotations:
        x, y, width, height = ann['bbox']

        # BBox가 subimg 영역과 겹치는지 확인
        if (x + width > x_offset and x < x_offset + subimg_width and
            y + height > y_offset and y < y_offset + subimg_height):
            
            # Update BBox coordinate
            new_x = max(x - x_offset, 0)
            new_y = max(y - y_offset, 0)
            width = min(width, x+width - x_offset)
            height = min(height, y+height - y_offset)

            updated_ann = ann.copy()
            updated_ann['bbox'] = [new_x, new_y, width, height]
            updated_ann['image_id'] = img_id
            updated_annotations.append(updated_ann)
    
    return updated_annotations

# Read annotation file
with open(annotation_path, 'r') as file:
    data = json.load(file)

# Define new images and annotations
new_images = []
new_annotations = []
new_img_id = max([img['id'] for img in data['images']]) + 1

# Load images
coco = COCO(annotation_path)
image_files = os.listdir(os.path.join(dataDir, 'train'))

for idx in tqdm(image_files, desc="Processing images"):
    img = coco.loadImgs(int(idx.split('_')[0]))[0]
    I = Image.open('{}/{}_x2_SR.png'.format(dataDir, img['file_name'].split('.')[0]))
    img_width, img_height = I.size

    # annotation ID
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)

    # 이미지를 4등분하는 영역 정의
    subimages = [
        (0, 0, img_width // 2, img_height // 2),  # 상단 왼쪽
        (img_width // 2, 0, img_width // 2, img_height // 2),  # 상단 오른쪽
        (0, img_height // 2, img_width // 2, img_height // 2),  # 하단 왼쪽
        (img_width // 2, img_height // 2, img_width // 2, img_height // 2)  # 하단 오른쪽
    ]

    # Update annotations for each partial image
    for i, subimg_info in enumerate(subimages):
        updated_anns = update_annotations_for_subimage(anns, subimg_info, new_img_id)

        # Draw X2 bounding box
        x_offset, y_offset, subimg_width, subimg_height = subimg_info
        subimg = I.crop((x_offset, y_offset, x_offset + subimg_width, y_offset + subimg_height))

        subimg_filename = '{}_{}_x2_SR.png'.format(img['file_name'].split('.')[0], i)
        # annotation file updated
        if updated_anns:
            new_img = {
                "width": subimg_width,
                "height": subimg_height,
                "file_name": subimg_filename,
                "date_captured": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "id": new_img_id
            }
            new_images.append(new_img)
            new_annotations.extend(updated_anns)

            # bbox가 있는 경우만 subimg 저장 
            subimg.save(os.path.join(subimgs_path, subimg_filename))
        
            new_img_id += 1

print("Updating annotation file...")

# 추가는 train.json 파일로 해야함
with open(original_anno_path, 'r') as file:
    original_data = json.load(file)

original_data['images'].extend(new_images)
original_data['annotations'].extend(new_annotations)

with open(updated_annotation_path, 'w') as file:
    json.dump(original_data, file, indent=2)

print("Annotation file updated successfully.")

### + 2048 X 2048 이미지에서 randomcrop 진행하고 json에 추가하기

In [ ]:
x2_annotation_path = '/data/ephemeral/home/dataset/train_x2_crop.json'

# Read json
with open(annotation_file, 'r') as file:
    data = json.load(file)

# Modify bbox values
for ann in data['annotations']:
    ann['bbox'][0] *= 2
    ann['bbox'][1] *= 2
    ann['bbox'][2] *= 2
    ann['bbox'][3] *= 2

# Store new json file
with open(x2_annotation_path, 'w') as file:
    json.dump(data, file, indent=2)

In [ ]:
# Dataset path
dataDir = '/data/ephemeral/home/EDSR-PyTorch/experiment/test/results-Demo'
annotation_path = '/data/ephemeral/home/dataset/train_x2_crop.json'
original_anno_path = '/data/ephemeral/home/dataset/train.json'

# saved path
subimgs_path = '/data/ephemeral/home/dataset/scale_x2_crop'
updated_annotation_path = '/data/ephemeral/home/dataset/train_x2_crop.json'

In [ ]:
def update_annotations_for_subimage_2(annotations, subimg_info, img_id):
    updated_annotations = []
    x_offset, y_offset, subimg_width, subimg_height = subimg_info

    for ann in annotations:
        x, y, width, height = ann['bbox']

        # BBox가 subimg 영역과 겹치는지 확인
        if (x + width > x_offset and x < x_offset + subimg_width and
            y + height > y_offset and y < y_offset + subimg_height):
            
            # Update BBox coordinate
            new_x = max(x - x_offset, 0)
            new_y = max(y - y_offset, 0)
            new_width = min(width, x + width - x_offset)
            new_height = min(height, y + height - y_offset)

            updated_ann = ann.copy()
            updated_ann['bbox'] = [new_x, new_y, new_width, new_height]
            updated_ann['image_id'] = img_id
            updated_annotations.append(updated_ann)
    
    return updated_annotations

In [ ]:
for idx in tqdm(image_files, desc="Processing images"):
    img = coco.loadImgs(int(idx.split('_')[0]))[0]
    I = Image.open('{}/{}_x2_SR.png'.format(dataDir, img['file_name'].split('.')[0]))
    img_width, img_height = I.size

    # annotation ID
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)

    # 기존에 처리한 4분할 이미지의 좌표 저장 (중복 방지)
    existing_crops = [
        (0, 0, img_width // 2, img_height // 2),
        (img_width // 2, 0, img_width // 2, img_height // 2),
        (0, img_height // 2, img_width // 2, img_height // 2),
        (img_width // 2, img_height // 2, img_width // 2, img_height // 2)
    ]

    # 1024x1024 크기로 랜덤 크롭 수행
    crop_size = (1024, 1024)
    if img_width >= 1024 and img_height >= 1024:
        for i in range(5):  # 한 이미지당 5개의 랜덤 크롭 생성
            random_crop_info = random_crop(I, crop_size, existing_crops)
            updated_anns = update_annotations_for_subimage_2(anns, random_crop_info, new_img_id)

            x_offset, y_offset, crop_width, crop_height = random_crop_info
            cropped_img = I.crop((x_offset, y_offset, x_offset + crop_width, y_offset + crop_height))

            cropped_filename = '{}_random_crop_{}_x2_SR.png'.format(img['file_name'].split('.')[0], i)
            if updated_anns:
                new_img = {
                    "width": crop_width,
                    "height": crop_height,
                    "file_name": cropped_filename,
                    "date_captured": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    "id": new_img_id
                }
                new_images.append(new_img)
                new_annotations.extend(updated_anns)

                cropped_img.save(os.path.join(subimgs_path, cropped_filename))
            
                new_img_id += 1

            # 새로 생성된 크롭 영역을 기존 크롭 리스트에 추가
            existing_crops.append(random_crop_info)

print("Updating annotation file...")

# 기존 데이터에 새로운 이미지와 어노테이션 추가
original_data = data.copy()
original_data['images'].extend(new_images)
original_data['annotations'].extend(new_annotations)

# 새로운 json 파일로 저장
with open('/data/ephemeral/home/dataset/train_x2_fixed_random.json', 'w') as file:
    json.dump(original_data, file, indent=2)

print("Annotation file updated successfully.")